## Lesson 23

GOAL: Create UNet finaly

In this lesson we will use it for super resolution

### Tiny Image net

64x64 images [tiny-imagenet](https://paperswithcode.com/dataset/tiny-imagenet)

* Notebook 24_imgnet_tiny.ipynb was discussed first.  (I note there are 3 other closly named notebooks though in the [repo](https://github.com/fastai/course22p2/tree/master))

* As in last week, my notes will be sparse as I don't intend to reproduce code here. I will just note the key points.

* TinyData set has labels like `n01443537` which are not human readable, they are some kind of encoded category. ('wordnet'). There are 200 categories. In the notebook they map these to integers 0-199.   There is also a words.txt file which turns these into human readable names.  Wordnet is some kind of hierarchical category system. 

* First part of the lesson focused just on setting up the data loaders for training and validation.

* Also data augmetnation was added, setting up random crop / horizinoal flip / and random erase.

#### Basic model 

* use the fashionmnist model with dropout (Resblocks) (Details in the notebook)
* Trains it for 25 epochs on classification task to get 60% accuracy (Training is at 91...so some overfitting)

#### Papers with code - how good can we do?

* [Tiny IMageNet Classification](https://paperswithcode.com/sota/image-classification-on-tiny-imagenet-1) shows results up to 93% on validation.  (Some of the models are using pretrained models, but at the current time it looks like the top performaer is not) 

* Highest resnet type model seems to get abotu 72% accruracy.  Oddly the paper link is missing now (was linked to the paper Jeremy looked at).  I found the paper was renamed to [Harnessing Hard Mixed Samples with Decoupled Regularizer](https://arxiv.org/abs/2203.10761)  Paper has to do with a proposal of new data augmentation technique. 

* Jeremy then makes a deeper model with multiple resblocks before downsampling. (i.e. multiple resblocks where there was one previously). This gets accuracy up to 62%.

*  Jeremy then adds *more* augmentation using something called [Trivial Augment](https://arxiv.org/abs/2103.10158) . This is parameter free and gives good results. It randomly picks a different augmentation for each image.  This is built into pytorch. `TrivialAugmentWide` is the version that is used.  Looking at teh source it picks from a wide range of augmentations, including Shear, Translate, Rotate, Brightness, etc. with random strength.  Unlike the others, this doesn't work well if the same transform is provided to all images in the batch.  So jeremy shows how to apply at a per item level. 

* Next he reorders the identiy as in   [Identity Mappings in Deep Residual Networks](https://arxiv.org/abs/1603.05027)  This is a small change  where the residual is added after the activation. This is called "Pre-activation" resblock beacuse the activation is applied before the residual is added. (This is supposed work better for *deeper* models).

* This results in a model that gets 65% accuracy after 50 epochs.  67.5% with 200 epochs.  This is basically the state of the art for these kinds of models


### Super Resolution
49 Minutes into the video

Notebook 25_superres.ipynb

* Independant variable will be a 32x32 scaled down image, and the dependent(label) variable will be the (original) 64x64 image.

* It is importatn that the transforms are the same for both the input and output. So they put it directly in the dataset class.

* To make the task a bit more difficult (regularization) they add a random erase only to the input (indpendant variable).   This forces the model to learn more about what the images should look like. ( The goal as a whole here is to get the model to figure out what pictures look like)

* To make it easier to set up for Unet, he downsamples the image to 32x32 and then interpolates it back up to 64x64. So the dimensions are the same but the image is 'blurry'.


#### First attempt, use an Autoencoder. 

* This is what we did near the beginning of the course (lesson 15, denoising was the goal then).  

* The model is a simple autoencoder with a few resblocks, and up_blocks wich use nn.UpsamplingNearest2d.   Remember this is meant to learn a higher order representation in the middle layers with very low resolution but a lot of feature (1024 in this case). 

* This doesn't work well...  Jeremy says it is because it is really difficult to learn a good representation in the middle layers. Maybe with a bigger model and longer training. 

* Instead, we will create a Unet... Finally ! 

#### Unet

* Original Unet paper is [U-Net: Convolutional Networks for Biomedical Image Segmentation](https://arxiv.org/abs/1505.04597)

* As with the autoencoder, we downsample and then upsample in the network to learn a latent representation. 

* The trick is to add skip connections from the downsampling to the upsampling. This is called "copy and crop" in the original paper. Here there will be no crop, but we will copy over the activations and concatenate (or add) to the upsapmpled activations.  This gives the later layers information on how the image should be layed out.  

* In the notebook Jeremy defines a TinyUnet model, using the same resblocks but adding the skip connections (using 'add'). In his version it looks like he has a direct skip from the input to the output

* For intialization, he wants to arrange it so the untrained model should just reproduce the output by zeroing out the last convolutions so that the last skip will just pass through to the output.  Or something like that. He seemed uncertain.

* This works MUCH better than the autoencoder.  Looks smooth, but doesn't get the detail of the original. Perhaps this is due to using MSE loss, which is just going to give the average if it doesn't know what to do.

#### Perceptual loss  
AT 1:16 minutes.

* The idea is to use the inner layers to compare the output to the target (i.e. in that level it can judge, hey there should be an eye here, etc).  

* Even though the classification accuracy was not great, the inner layers are still good at detecting features.   Remember this model was just a bunch of resenets.  So jeremy takes this model and deletes the end layers, keeping only the first 3 resblocks.

* This `cmodel` will take a 64x64 image and output an 1024x8x8 tensor.  We can use this compute the perceptual loss by comparing (mse) the cmodel of the target and the u-net output.  This can be combined iwth the normal loss with some weights (he uses 0.1 but this is a hyperparameter).

*  Results: a bit better... less blurry looking anyway. 


#### Initialize with pretrained model

* Use the tiny imagenet classification model weights to initialize the downsample part of the unet. 
* You use the state_dict to do this.
* THen freeze the downsample part of the mode.  
* Do one epoch of training and we get better then we did before! This is cheating but it is fine, this is just what we want!
* Now we can train the whole model with a better starting point.
* Result? Even better. Most images look pretty good.


#### Cross Convolution
1:34 minutes

* Try adding some resblocks along the skip connections so that the model can learn a bit better what to do with the skip connections.

* Comparing the results - it gets better loss, and NOW the  Koala has an eye! (It didn't before).

#### Diffusion Unet 

NEXT TIME!


### 